In [1]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

import numpy as np

In [2]:
model = YOLO("yolo11n-pose.pt")
# model.export(
#     format="engine",
#     imgsz=[256, 448],
#     # dynamic=True,
#     # simplify=True,
#     half=True,
#     # int8=True,
# )
# tensorrt_model = YOLO("yolo11n-pose.engine")

# self.model = YOLO("yolo11n-pose.pt")
model.export(
    format="engine",
    imgsz=[256, 448],
    half=True,
)
tensorrt_model = YOLO("yolo11n-pose.engine")

WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics 8.3.130 🚀 Python-3.10.12 torch-2.3.0 CUDA:0 (Orin, 7620MiB)
YOLO11n-pose summary (fused): 109 layers, 2,866,468 parameters, 0 gradients, 7.4 GFLOPs

PyTorch: starting from 'yolo11n-pose.pt' with input shape (1, 3, 256, 448) BCHW and output shape(s) (1, 56, 2352) (6.0 MB)

ONNX: starting export with onnx 1.17.0 opset 17...
ONNX: slimming with onnxslim 0.1.52...
ONNX: export success ✅ 4.5s, saved as 'yolo11n-pose.onnx' (11.1 MB)

TensorRT: starting export with TensorRT 8.6.2...
[05/12/2025-20:59:45] [TRT] [I] [MemUsageChange] Init CUDA: CPU +2, GPU +0, now: CPU 705, GPU 6299 (MiB)
[05/12/2025-20:59:52] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +1154, GPU +687, now: CPU 1895, GPU 7010 (MiB)
[05/12/2025-20:59:53] [TRT] [I] ----------------------------------------------------------------
[05/12/2025-20:59:53] [TRT] [I] Input filename:   yolo11n-pose.onnx
[05/12/2025-20:59:53] [TRT] [I] ON

In [7]:
img = cv2.imread("./bus.jpg")
img_ = cv2.resize(img, (448, 256))

In [10]:
img_.shape

(256, 448, 3)

In [8]:
%timeit model(img_, verbose=False)

41.9 ms ± 618 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%timeit tensorrt_model(img, imgsz=[256, 448], verbose=False)

14.6 ms ± 155 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
skeleton = [
    [16, 14],
    [14, 12],
    [17, 15],
    [15, 13],
    [12, 13],
    [6, 12],
    [7, 13],
    [6, 7],
    [6, 8],
    [7, 9],
    [8, 10],
    [9, 11],
    [2, 3],
    [1, 2],
    [1, 3],
    [2, 4],
    [3, 5],
    [4, 6],
    [5, 7],
]
pose_palette = np.array([
    [255, 128, 0],
    [255, 153, 51],
    [255, 178, 102],
    [230, 230, 0],
    [255, 153, 255],
    [153, 204, 255],
    [255, 102, 255],
    [255, 51, 255],
    [102, 178, 255],
    [51, 153, 255],
    [255, 153, 153],
    [255, 102, 102],
    [255, 51, 51],
    [153, 255, 153],
    [102, 255, 102],
    [51, 255, 51],
    [0, 255, 0],
    [0, 0, 255],
    [255, 0, 0],
    [255, 255, 255],
], dtype=np.uint8)

limb_color = pose_palette[[9, 9, 9, 9, 7, 7, 7, 0, 0, 0, 0, 0, 16, 16, 16, 16, 16, 16, 16]]
kpt_color = pose_palette[[16, 16, 16, 16, 16, 0, 0, 0, 0, 0, 0, 9, 9, 9, 9, 9, 9]]

conf_thres = 0.5
result = results[0]
img = cv2.imread("./bus.jpg")
for bbox, kpts, confs in zip(result.boxes.xywh, result.keypoints.xy, result.keypoints.conf):
    x, y, w, h = map(int, bbox)
    cv2.rectangle(img, (x-w//2, y-h//2), (x+w//2, y+h//2), (0, 255, 0), 2)
    
    for j, (kpt, conf) in enumerate(zip(kpts, confs)):
        if conf < conf_thres:
            continue

        x, y = int(kpt[0]), int(kpt[1])
        cv2.circle(img, (x, y), 5, kpt_color[j].tolist(), -1)
    
    for i, sk in enumerate(skeleton):
        pos1 = (int(kpts[(sk[0] - 1), 0]), int(kpts[(sk[0] - 1), 1]))
        pos2 = (int(kpts[(sk[1] - 1), 0]), int(kpts[(sk[1] - 1), 1]))

        conf1 = confs[(sk[0] - 1)]
        conf2 = confs[(sk[1] - 1)]
        if conf1 < conf_thres or conf2 < conf_thres:
            continue
        # if pos1[0] % shape[1] == 0 or pos1[1] % shape[0] == 0 or pos1[0] < 0 or pos1[1] < 0:
        #     continue
        # if pos2[0] % shape[1] == 0 or pos2[1] % shape[0] == 0 or pos2[0] < 0 or pos2[1] < 0:
        #     continue
        cv2.line(
            img,
            pos1, pos2,
            limb_color[i].tolist(),
            thickness=3,
            lineType=cv2.LINE_AA,
        )


In [ ]:
plt.imshow(img)

In [ ]:
results[0].keypoints.conf

In [ ]:
len(results[0].boxes)